**P28** Queremos saber donde vive alguno de nuestros usuarios, para los usuarios que tiene más de 50 registros a negocios distintos en la tabla reviews obtenga el promedio y desviación estándar de la latitud y longitud de los negocios que calificaron (contando cada negocio una sola vez). Para el usuario que menos desviación estándar sumada tenga de ambas coordenadas muestre ese promedio y dónde está eso (https://www.gps-coordinates.net/) y cómo se llama el usuario para después irlo a buscar a la casa.

In [1]:
!pip install pandas==1.4.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 23.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 1.4.2 which is incompatible.


In [2]:
import pandas as pd

In [3]:
# Change the file directories to yours, the files are in a public folder in google drive:
# https://drive.google.com/drive/folders/1CsvJs0xZ9SCLtG4ci2dNGCkhqnI-1DLS?usp=drive_link
df_business = pd.read_csv('/content/drive/MyDrive/2022/Orga_de_Datos/Datasets/business.csv', usecols=['business_id', 'latitude', 'longitude'])
df_review = pd.read_csv('/content/drive/MyDrive/2022/Orga_de_Datos/Datasets/review.csv', usecols=['user_id', 'business_id'])
df_user = pd.read_csv('/content/drive/MyDrive/2022/Orga_de_Datos/Datasets/user.csv')

In [4]:
df_user = df_user.set_index('user_id')
df_business = df_business.set_index('business_id')

In [5]:
df_unique_reviews = df_review.drop_duplicates()
df_unique_reviews = df_unique_reviews.groupby(['user_id']).agg({'business_id': lambda x: x.tolist()})
df_unique_reviews.columns = ['unique_reviews']
df_unique_reviews.dropna(inplace=True)

In [6]:
useful_user_reviews = df_unique_reviews[df_unique_reviews['unique_reviews'].map(len) > 50]

In [7]:
def col_mean(x, column_name):
  lista = []
  for business in x['unique_reviews']:
    lista.append(df_business.loc[business][column_name])
  return pd.Series(lista).mean()

def col_std(x, column_name):
  lista = []
  for business in x['unique_reviews']:
    lista.append(df_business.loc[business][column_name])
  return pd.Series(lista).std()

useful_user_reviews['avg_lat'] = useful_user_reviews.apply(col_mean, column_name='latitude', axis=1)
useful_user_reviews['avg_lon'] = useful_user_reviews.apply(col_mean, column_name='longitude', axis=1)
useful_user_reviews['std_lat'] = useful_user_reviews.apply(col_std, column_name='latitude', axis=1)
useful_user_reviews['std_lon'] = useful_user_reviews.apply(col_std, column_name='longitude', axis=1)
useful_user_reviews['std_sum'] = useful_user_reviews[['std_lat','std_lon']].sum(axis=1)

<ipython-input-7-3e8cc8f08ca6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_user_reviews['avg_lat'] = useful_user_reviews.apply(col_mean, column_name='latitude', axis=1)
<ipython-input-7-3e8cc8f08ca6>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_user_reviews['avg_lon'] = useful_user_reviews.apply(col_mean, column_name='longitude', axis=1)
<ipython-input-7-3e8cc8f08ca6>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [8]:
final_user = useful_user_reviews.sort_values(by='std_sum').head(1)

In [9]:
lat = final_user['avg_lat'].values[0]
lon = final_user['avg_lon'].values[0]
name = df_user.loc[final_user.index[0]].values[0]

print(f'the user lives in lat: {lat} lon: {lon} and his/her name is: {name}')

the user lives in lat: 39.947476974315 lon: -75.16780540930333 and his/her name is: Nhu
